# GameTheory 21 - Jeux Cooperatifs en Lean : Formalisation de Shapley

**Navigation** : [<< 15-CooperativeGames (track principal)]([15-CooperativeGames](GameTheory-15-CooperativeGames.ipynb)) | [Index](README.md)

**Autres side tracks** : [15c-CooperativeGames-Python](GameTheory-15c-CooperativeGames-Python.ipynb)

**Kernel** : Lean 4 (WSL)

**Notebook Python compagnon** : [GameTheory-15c-CooperativeGames-Python](GameTheory-15c-CooperativeGames-Python.ipynb) (calculs numeriques, exemples)

---

## Configuration du projet Lake avec Mathlib

Ce notebook utilise **Mathlib** pour les types mathematiques (`Finset`, `Fintype`, `R`) et les tactiques avancees. Un projet Lake a ete configure dans `cooperative_games_lean/`.

### Option 1 : Executer avec le projet Lake (recommande)

Pour executer le code de ce notebook avec Mathlib :

```bash
# 1. Aller dans le repertoire du projet Lake
cd MyIA.AI.Notebooks/GameTheory/cooperative_games_lean

# 2. Telecharger le cache Mathlib pre-compile (~100 MB, 1-2 min)
lake exe cache get

# 3. Construire le projet
lake build

# 4. Executer du code Lean dans le contexte du projet
lake env lean --run CooperativeGames.lean
```

### Option 2 : Mode pedagogique (ce notebook)

Ce notebook presente la **structure logique** du code Lean. Sans la configuration Lake, certaines cellules afficheront des erreurs dues aux imports Mathlib manquants - c'est normal ! L'objectif est de comprendre **comment** formaliser ces concepts.

### Fichiers Lean du projet

| Fichier | Contenu |
|---------|---------|
| `cooperative_games_lean/CooperativeGames.lean` | Module principal |
| `cooperative_games_lean/CooperativeGames/Basic.lean` | TUGame, Core, convexite |
| `cooperative_games_lean/CooperativeGames/Shapley.lean` | Axiomes, valeur, unicite |

---

## Introduction

Ce notebook formalise les **jeux cooperatifs** et la **valeur de Shapley** en Lean 4. Ces concepts, introduits dans le notebook 14 (Python), sont ici traites avec la rigueur des preuves formelles.

### Contenu

1. **Jeux cooperatifs (TU games)** : Fonction caracteristique v(S)
2. **Axiomes de Shapley** : Efficacite, symetrie, additivite, joueur nul
3. **Valeur de Shapley** : Definition et unicite
4. **Le Core** : Stabilite des allocations
5. **Jeux de vote** : Indice de Banzhaf

### Lien avec les notebooks precedents

| Notebook | Contenu |
|----------|--------|
| **14 (Python)** | Implementation : calcul exact/Monte Carlo, exemple politique |
| **20 (Lean)** | Social choice : Arrow, Sen, electeur median |
| **21 (Lean)** | Formalisation : axiomes, theoreme d'unicite de Shapley |

### Duree estimee : 60 minutes

---

## 1. Jeux Cooperatifs (TU Games)

### 1.1 Definition mathematique

Un **jeu cooperatif a utilite transferable** (TU game) est defini par :
- Un ensemble fini de joueurs N = {1, 2, ..., n}
- Une **fonction caracteristique** v : 2^N -> R avec v(empty) = 0

La valeur v(S) represente ce que la coalition S peut obtenir en cooperant, independamment de ce que font les joueurs hors de S.

> **Note Lean** : Le code ci-dessous utilise `Fintype N` (ensemble fini de joueurs) et `Finset N` (sous-ensembles finis) de Mathlib. La structure `TUGame` encode la fonction caracteristique avec sa contrainte `v(empty) = 0`.

In [1]:
-- Jeu cooperatif a utilite transferable (TU game)

-- Type des joueurs (fini)
variable {N : Type*} [Fintype N] [DecidableEq N]

-- Coalition = sous-ensemble de joueurs
def Coalition (N : Type*) := Finset N

-- Fonction caracteristique : v(S) donne la valeur d'une coalition
structure TUGame (N : Type*) [Fintype N] where
  v : Finset N → ℝ
  empty_zero : v ∅ = 0

#check TUGame
#check @TUGame.v

-- Jeu cooperatif a utilite transferable (TU game)

-- Type des joueurs (fini)
variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Coalition = sous-ensemble de joueurs
def Coalition (N : Type*) := Finset N
                        ─▶ ❌ unexpected token ')'; expected term

-- Fonction caracteristique : v(S) donne la valeur d'une coalition
structure TUGame (N : Type*) [Fintype N] where
                           ─▶ ❌ unexpected token ')'; expected term
  v : Finset N → ℝ
  empty_zero : v ∅ = 0

#check TUGame
       ──────▶ ❌ Unknown identifier `TUGame`
#check @TUGame.v
        ────────▶ ❌ Unknown identifier `TUGame.v`
--% env 0

Raw input:
{"cmd": "-- Jeu cooperatif a utilite transferable (TU game)\n\n-- Type des joueurs (fini)\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Coalition = sous-ensemble de joueurs\ndef Coalition (N : Type*) := Finset N\n\n-- Fonction caracteristique : v(S) donne la valeur d'une coalition\nstructure TUGame (N : Type*) [Fintype N] where\n  v : Finset N \u2192 \u211d\n  empty_zero : v \u2205 = 0\n\n#check TUGame\n#check @TUGame.v"}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 4, "column": 19},
   "endPos": {"line": 4, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 7, "column": 24},
   "endPos": {"line": 7, "column": 25},
   "data": "unexpected token ')'; expected term"},
  {"severity": "error",
   "pos": {"line": 10, "column": 27},
   "endPos": {"line": 10, "column": 28},
   "data": "unexpected token ')'; expected term"},
  {"severity": "error",
   "pos": {"line": 14, "column": 7},
   "endPos": {"line": 14, "column": 13},
   "data": "Unknown identifier `TUGame`"},
  {"severity": "error",
   "pos": {"line": 15, "column": 8},
   "endPos": {"line": 15, "column": 16},
   "data": "Unknown identifier `TUGame.v`"}],
 "env": 0}

### 1.2 Proprietes structurelles

In [2]:
-- Proprietes des jeux cooperatifs

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Superadditivite : la cooperation est benefique
-- v(S ∪ T) ≥ v(S) + v(T) pour S, T disjoints
def Superadditive (G : TUGame N) : Prop :=
  ∀ S T : Finset N, Disjoint S T → 
    G.v (S ∪ T) ≥ G.v S + G.v T

-- Convexite (supermodularite) : contributions marginales croissantes
-- v(S ∪ {i}) - v(S) ≤ v(T ∪ {i}) - v(T) pour S ⊆ T, i ∉ T
def Convex (G : TUGame N) : Prop :=
  ∀ S T : Finset N, ∀ i : N, 
    S ⊆ T → i ∉ T →
    G.v (S ∪ {i}) - G.v S ≤ G.v (T ∪ {i}) - G.v T

#check @Superadditive
#check @Convex

-- Proprietes des jeux cooperatifs

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Superadditivite : la cooperation est benefique
-- v(S ∪ T) ≥ v(S) + v(T) pour S, T disjoints
def Superadditive (G : TUGame N) : Prop :=
                       ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S T : Finset N, Disjoint S T → 
          ──────▶ ❌ Unknown identifier `Finset`
          ──────▶ ❌ Unknown identifier `Finset`
                    ────────▶ ❌ Unknown identifier `Disjoint`
    G.v (S ∪ T) ≥ G.v S + G.v T

-- Convexite (supermodularite) : contributions marginales croissantes
-- v(S ∪ {i}) - v(S) ≤ v(T ∪ {i}) - v(T) pour S ⊆ T, i ∉ T
def Convex (G : TUGame N) : Prop :=
                ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S T : Finset N, ∀ i : N, 
          ──────▶ ❌ Unknown identifier `Finset`
          ──────▶ ❌ Unknown identifier `Finset`
                          ─▶ ❌ Unknown identifier `N`
    S ⊆ T → i ∉ T →
    G.v (S ∪ {i}) - G.v S ≤ G.v (T ∪ {i}) - G.v T

#check @Superadditive
──────▶  @Superadditive : {x : Sort u_1} → {TUGame : x} → sorry → Prop
#check @Convex
──────▶  @Convex : {x : Sort u_1} → {TUGame : x} → sorry → Prop
--% env 1

Raw input:
{"cmd": "-- Proprietes des jeux cooperatifs\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Superadditivite : la cooperation est benefique\n-- v(S \u222a T) \u2265 v(S) + v(T) pour S, T disjoints\ndef Superadditive (G : TUGame N) : Prop :=\n  \u2200 S T : Finset N, Disjoint S T \u2192 \n    G.v (S \u222a T) \u2265 G.v S + G.v T\n\n-- Convexite (supermodularite) : contributions marginales croissantes\n-- v(S \u222a {i}) - v(S) \u2264 v(T \u222a {i}) - v(T) pour S \u2286 T, i \u2209 T\ndef Convex (G : TUGame N) : Prop :=\n  \u2200 S T : Finset N, \u2200 i : N, \n    S \u2286 T \u2192 i \u2209 T \u2192\n    G.v (S \u222a {i}) - G.v S \u2264 G.v (T \u222a {i}) - G.v T\n\n#check @Superadditive\n#check @Convex", "env": 0}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 7, "column": 23},
   "endPos": {"line": 7, "column": 31},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement."},
  {"severity": "error",
   "pos": {"line": 8, "column": 10},
   "endPos": {"line": 8, "column": 16},
   "data": "Unknown identifier `Finset`"},
  {"severity": "error",
   "pos": {"line": 8, "column": 10},
   "endPos": {"line": 8, "column": 16},
   "data": "Unknown identifier `Finset`"},
  {"severity": "erro

### 1.3 Exemples classiques

In [3]:
-- Exemples de jeux cooperatifs

-- Jeu d'unanimite : v(S) = 1 si T ⊆ S, 0 sinon
-- Les joueurs de T doivent tous etre presents pour creer de la valeur
def unanimityGame {N : Type*} [Fintype N] [DecidableEq N] 
    (T : Finset N) : TUGame N := {
  v := fun S => if T ⊆ S then 1 else 0
  empty_zero := by
    simp only
    by_cases hT : T = ∅
    · simp [hT]
    · simp only [Finset.not_subset_empty_iff.mpr hT, ite_false]
}

-- Jeu de majorite simple : v(S) = 1 si |S| > n/2, 0 sinon
def majorityGame {N : Type*} [Fintype N] [DecidableEq N] : TUGame N := {
  v := fun S => if 2 * S.card > Fintype.card N then 1 else 0
  empty_zero := by simp
}

#check @unanimityGame
#check @majorityGame

-- Exemples de jeux cooperatifs

-- Jeu d'unanimite : v(S) = 1 si T ⊆ S, 0 sinon
-- Les joueurs de T doivent tous etre presents pour creer de la valeur
def unanimityGame {N : Type*} [Fintype N] [DecidableEq N] 
                            ─▶ ❌ unexpected token '}'; expected term
    (T : Finset N) : TUGame N := {
  v := fun S => if T ⊆ S then 1 else 0
  empty_zero := by
    simp only
    by_cases hT : T = ∅
    · simp [hT]
    · simp only [Finset.not_subset_empty_iff.mpr hT, ite_false]
}

-- Jeu de majorite simple : v(S) = 1 si |S| > n/2, 0 sinon
def majorityGame {N : Type*} [Fintype N] [DecidableEq N] : TUGame N := {
                           ─▶ ❌ unexpected token '}'; expected term
  v := fun S => if 2 * S.card > Fintype.card N then 1 else 0
  empty_zero := by simp
}

#check @unanimityGame
        ─────────────▶ ❌ Unknown identifier `unanimityGame`
#check @majorityGame
        ────────────▶ ❌ Unknown identifier `majorityGame`
--% env 2

Raw input:
{"cmd": "-- Exemples de jeux cooperatifs\n\n-- Jeu d'unanimite : v(S) = 1 si T \u2286 S, 0 sinon\n-- Les joueurs de T doivent tous etre presents pour creer de la valeur\ndef unanimityGame {N : Type*} [Fintype N] [DecidableEq N] \n    (T : Finset N) : TUGame N := {\n  v := fun S => if T \u2286 S then 1 else 0\n  empty_zero := by\n    simp only\n    by_cases hT : T = \u2205\n    \u00b7 simp [hT]\n    \u00b7 simp only [Finset.not_subset_empty_iff.mpr hT, ite_false]\n}\n\n-- Jeu de majorite simple : v(S) = 1 si |S| > n/2, 0 sinon\ndef majorityGame {N : Type*} [Fintype N] [DecidableEq N] : TUGame N := {\n  v := fun S => if 2 * S.card > Fintype.card N then 1 else 0\n  empty_zero := by simp\n}\n\n#check @unanimityGame\n#check @majorityGame", "env": 1}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 5, "column": 28},
   "endPos": {"line": 5, "column": 29},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 16, "column": 27},
   "endPos": {"line": 16, "column": 28},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 21, "column": 8},
   "endPos": {"line": 21, "column": 21},
   "data": "Unknown identifier `unanimityGame`"},
  {"severity": "error",
   "pos": {"line": 22, "column": 8},
   "endPos": {"line": 22, "column": 20},
   "data": "Unknown identifier `majorityGame`"}],
 "env": 2}

---

## 2. Axiomes de Shapley

Shapley (1953) a montre que quatre axiomes caracterisent de maniere unique la valeur de Shapley :

1. **Efficacite** : La somme des paiements egale la valeur de la grande coalition
2. **Symetrie** : Joueurs interchangeables recoivent le meme paiement
3. **Joueur nul** : Un joueur sans contribution marginale recoit 0
4. **Additivite** : La valeur de la somme de deux jeux = somme des valeurs

In [4]:
-- Axiomes de Shapley pour une solution φ

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une solution : associe a chaque jeu un vecteur de paiements
def Solution (N : Type*) [Fintype N] := TUGame N → (N → ℝ)

-- Axiome 1 : Efficacite
def Efficiency (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∑ i, φ G i = G.v Finset.univ

-- Axiome 2 : Symetrie
def Symmetric (G : TUGame N) (i j : N) : Prop :=
  ∀ S : Finset N, i ∉ S → j ∉ S →
    G.v (S ∪ {i}) = G.v (S ∪ {j})

def SymmetryAxiom (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∀ i j : N, 
    Symmetric G i j → φ G i = φ G j

-- Axiome 3 : Joueur nul
def NullPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, i ∉ S → 
    G.v (S ∪ {i}) = G.v S

def NullPlayerAxiom (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∀ i : N,
    NullPlayer G i → φ G i = 0

-- Axiome 4 : Additivite
def AddGames (G H : TUGame N) : TUGame N := {
  v := fun S => G.v S + H.v S
  empty_zero := by simp [G.empty_zero, H.empty_zero]
}

def AdditivityAxiom (φ : Solution N) : Prop :=
  ∀ G H : TUGame N, ∀ i : N,
    φ (AddGames G H) i = φ G i + φ H i

#check @Efficiency
#check @SymmetryAxiom
#check @NullPlayerAxiom
#check @AdditivityAxiom

-- Axiomes de Shapley pour une solution φ

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Une solution : associe a chaque jeu un vecteur de paiements
def Solution (N : Type*) [Fintype N] := TUGame N → (N → ℝ)
                       ─▶ ❌ unexpected token ')'; expected term

-- Axiome 1 : Efficacite
def Efficiency (φ : Solution N) : Prop :=
                    ──────────▶ ❌ Function expected at
  Solution
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Solution` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ G : TUGame N, ∑ i, φ G i = G.v Finset.univ
──────────────────▶ ❌ expected token

-- Axiome 2 : Symetrie
def Symmetric (G : TUGame N) (i j : N) : Prop :=
                   ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S : Finset N, i ∉ S → j ∉ S →
        ──────▶ ❌ Unknown identifier `Finset`
    G.v (S ∪ {i}) = G.v (S ∪ {j})

def SymmetryAxiom (φ : Solution N) : Prop :=
                       ──────────▶ ❌ Function expected at
  Solution
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Solution` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ G : TUGame N, ∀ i j : N, 
        ──────▶ ❌ Unknown identifier `TUGame`
                          ─▶ ❌ Unknown identifier `N`
                          ─▶ ❌ Unknown identifier `N`
    Symmetric G i j → φ G i = φ G j

-- Axiome 3 : Joueur nul
def NullPlayer (G : TUGame N) (i : N) : Prop :=
                    ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S : Finset N, i ∉ S → 
        ──────▶ ❌ Unknown identifier `Finset`
    G.v (S ∪ {i}) = G.v S

def NullPlayerAxiom (φ : Solution N) : Prop :=
                         ──────────▶ ❌ Function expected at
  Solution
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Solution` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ G : TUGame N, ∀ i : N,
        ──────▶ ❌ Unknown identifier `TUGame`
                        ─▶ ❌ Unknown identifier `N`
    NullPlayer G i → φ G i = 0

-- Axiome 4 : Additivite
def AddGames (G H : TUGame N) : TUGame N := {
       

---

## 3. Valeur de Shapley

### 3.1 Definition

La valeur de Shapley du joueur $i$ est sa contribution marginale moyenne sur toutes les permutations :

$$\phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(n-|S|-1)!}{n!} [v(S \cup \{i\}) - v(S)]$$

In [5]:
-- Definition de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Contribution marginale de i a la coalition S
def marginalContribution (G : TUGame N) (i : N) (S : Finset N) : ℝ :=
  G.v (S ∪ {i}) - G.v S

-- Coefficient de Shapley pour une coalition de taille s sur n joueurs
noncomputable def shapleyCoef (n s : ℕ) : ℝ :=
  (Nat.factorial s * Nat.factorial (n - s - 1)) / Nat.factorial n

-- Valeur de Shapley
noncomputable def shapleyValue (G : TUGame N) (i : N) : ℝ :=
  ∑ S in (Finset.univ.filter (fun S => i ∉ S)),
    shapleyCoef (Fintype.card N) S.card * marginalContribution G i S

#check @marginalContribution
#check @shapleyValue

-- Definition de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Contribution marginale de i a la coalition S
def marginalContribution (G : TUGame N) (i : N) (S : Finset N) : ℝ :=
                              ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
                                                     ────────▶ ❌ Function expected at
  Finset
but this term has type
  ?m.3

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Finset` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  G.v (S ∪ {i}) - G.v S

-- Coefficient de Shapley pour une coalition de taille s sur n joueurs
noncomputable def shapleyCoef (n s : ℕ) : ℝ :=
  (Nat.factorial s * Nat.factorial (n - s - 1)) / Nat.factorial n
   ─────────────▶ ❌ Unknown constant `Nat.factorial`
                     ─────────────▶ ❌ Unknown constant `Nat.factorial`
                                                  ─────────────▶ ❌ Unknown constant `Nat.factorial`

-- Valeur de Shapley
noncomputable def shapleyValue (G : TUGame N) (i : N) : ℝ :=
                                    ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∑ S in (Finset.univ.filter (fun S => i ∉ S)),
──▶ ❌ expected token
    shapleyCoef (Fintype.card N) S.card * marginalContribution G i S

#check @marginalContribution
──────▶  @marginalContribution : {x : Sort u_1} →
  {TUGame : x} → {N : Sort u_2} → {x : Sort u_3} → {Finset : x} → {ℝ : Type u_4} → sorry → N → sorry → ℝ
#check @shapleyValue
──────▶  @shapleyValue : {x : Sort u_1} → {TUGame : x} → {N : Sort u_2} → {ℝ : Sort u_3} → sorry → N → ℝ
--% env 4

Raw input:
{"cmd": "-- Definition de la valeur de Shapley\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Contribution marginale de i a la coalition S\ndef marginalContribution (G : TUGame N) (i : N) (S : Finset N) : \u211d :=\n  G.v (S \u222a {i}) - G.v S\n\n-- Coefficient de Shapley pour une coalition de taille s sur n joueurs\nnoncomputable def shapleyCoef (n s : \u2115) : \u211d :=\n  (Nat.factorial s * Nat.factorial (n - s - 1)) / Nat.factorial n\n\n-- Valeur de Shapley\nnoncomputable def shapleyValue (G : TUGame N) (i : N) : \u211d :=\n  \u2211 S in (Finset.univ.filter (fun S => i \u2209 S)),\n    shapleyCoef (Fintype.card N) S.card * marginalContribution G i S\n\n#check @marginalContribution\n#check @shapleyValue", "env": 3}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 6, "column": 30},
   "endPos": {"line": 6, "column": 38},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because

### 3.2 Theoreme d'unicite

In [6]:
-- Theoreme de Shapley : la valeur de Shapley est l'unique solution
-- satisfaisant les quatre axiomes

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem shapley_uniqueness_sketch (φ : Solution N)
    (h_eff : Efficiency φ)
    (h_sym : SymmetryAxiom φ)
    (h_null : NullPlayerAxiom φ)
    (h_add : AdditivityAxiom φ) :
    -- Alors φ est la valeur de Shapley
    ∀ G : TUGame N, ∀ i : N, φ G i = shapleyValue G i := by
  -- La preuve utilise la decomposition en jeux d'unanimite
  -- G = ∑_T c_T * u_T
  -- Par additivite et les autres axiomes
  sorry

#check @shapley_uniqueness_sketch

-- Theoreme de Shapley : la valeur de Shapley est l'unique solution
-- satisfaisant les quatre axiomes

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

theorem shapley_uniqueness_sketch (φ : Solution N)
                                       ──────────▶ ❌ Function expected at
  Solution
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Solution` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    (h_eff : Efficiency φ)
    (h_sym : SymmetryAxiom φ)
    (h_null : NullPlayerAxiom φ)
    (h_add : AdditivityAxiom φ) :
    -- Alors φ est la valeur de Shapley
    ∀ G : TUGame N, ∀ i : N, φ G i = shapleyValue G i := by
          ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.2

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  -- La preuve utilise la decomposition en jeux d'unanimite
  -- G = ∑_T c_T * u_T
  -- Par additivite et les autres axiomes
  sorry

#check @shapley_uniqueness_sketch
        ─────────────────────────▶ ❌ Unknown identifier `shapley_uniqueness_sketch`
--% env 5
--% prove 0

Raw input:
{"cmd": "-- Theoreme de Shapley : la valeur de Shapley est l'unique solution\n-- satisfaisant les quatre axiomes\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\ntheorem shapley_uniqueness_sketch (\u03c6 : Solution N)\n    (h_eff : Efficiency \u03c6)\n    (h_sym : SymmetryAxiom \u03c6)\n    (h_null : NullPlayerAxiom \u03c6)\n    (h_add : AdditivityAxiom \u03c6) :\n    -- Alors \u03c6 est la valeur de Shapley\n    \u2200 G : TUGame N, \u2200 i : N, \u03c6 G i = shapleyValue G i := by\n  -- La preuve utilise la decomposition en jeux d'unanimite\n  -- G = \u2211_T c_T * u_T\n  -- Par additivite et les autres axiomes\n  sorry\n\n#check @shapley_uniqueness_sketch", "env": 4}
Raw output:
{"sorries":
 [{"proofState": 0,
   "pos": {"line": 16, "column": 2},
   "goal":
   "x✝¹ : Sort u_1\nSolution : x✝¹\nx✝ : Sort u_2\nTUGame : x✝\nN : Sort u_3\nφ : sorry\nh_eff : Efficiency φ\nh_sym : SymmetryAxiom φ\nh_null : NullPlayerAxiom φ\nh_add : AdditivityAxiom φ\n⊢ ∀ (G : sorry) (i : N), ?m.19 = shapleyValue G i",
   "endPos": {"line": 16, "column": 7}}],
 "messages":
 [{"severity": "error",
   "pos": {"line": 4, "column": 19},
   "endPos": {"line": 4, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 6, "column": 39},
   "endPos": {"line": 6, "column": 49},
   "data":
   "Function expected at\n  Solution\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `Solution` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement."},
  {"severity": "error",
   "pos": {"line": 12, "column": 10},
   "endPos": {"line": 12, "column": 18},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.2\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `TUGame` is unknown, and Lean

### 3.3 Proprietes de la valeur de Shapley

In [7]:
-- Proprietes de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- La valeur de Shapley satisfait l'efficacite
theorem shapley_efficient : ∀ G : TUGame N,
    ∑ i, shapleyValue G i = G.v Finset.univ := by
  sorry

-- Pour les jeux d'unanimite, la valeur a une forme simple
theorem shapley_unanimity (T : Finset N) (i : N) :
    shapleyValue (unanimityGame T) i = 
    if i ∈ T then (1 : ℝ) / T.card else 0 := by
  sorry

#check @shapley_efficient
#check @shapley_unanimity

-- Proprietes de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- La valeur de Shapley satisfait l'efficacite
theorem shapley_efficient : ∀ G : TUGame N,
    ∑ i, shapleyValue G i = G.v Finset.univ := by
────▶ ❌ expected token
  sorry

-- Pour les jeux d'unanimite, la valeur a une forme simple
theorem shapley_unanimity (T : Finset N) (i : N) :
                               ────────▶ ❌ Function expected at
  Finset
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Finset` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    shapleyValue (unanimityGame T) i = 
                  ───────────────▶ ❌ Function expected at
  unanimityGame
but this term has type
  ?m.3

Note: Expected a function because this term is being applied to the argument
  T

Hint: The identifier `unanimityGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    if i ∈ T then (1 : ℝ) / T.card else 0 := by
                   ─▶ ❌ failed to synthesize instance of type class
  OfNat ℝ 1
numerals are polymorphic in Lean, but the numeral `1` cannot be used in a context where the expected type is
  ℝ
due to the absence of the instance above

Hint: Type class instance resolution failures can be inspected with the `set_option trace.Meta.synthInstance true` command.
  sorry

#check @shapley_efficient
        ─────────────────▶ ❌ Unknown identifier `shapley_efficient`
#check @shapley_unanimity
        ─────────────────▶ ❌ Unknown identifier `shapley_unanimity`
--% env 6
--% prove 1

Raw input:
{"cmd": "-- Proprietes de la valeur de Shapley\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- La valeur de Shapley satisfait l'efficacite\ntheorem shapley_efficient : \u2200 G : TUGame N,\n    \u2211 i, shapleyValue G i = G.v Finset.univ := by\n  sorry\n\n-- Pour les jeux d'unanimite, la valeur a une forme simple\ntheorem shapley_unanimity (T : Finset N) (i : N) :\n    shapleyValue (unanimityGame T) i = \n    if i \u2208 T then (1 : \u211d) / T.card else 0 := by\n  sorry\n\n#check @shapley_efficient\n#check @shapley_unanimity", "env": 5}
Raw output:
{"sorries":
 [{"proofState": 1,
   "pos": {"line": 14, "column": 2},
   "goal":
   "x✝¹ : Sort u_1\nFinset : x✝¹\nN : Type u_2\nx✝ : Sort u_3\nunanimityGame : x✝\nℝ : Type u_4\nT : sorry\ni : N\n⊢ shapleyValue sorry i = if i ∈ T then sorry / sorry else 0",
   "endPos": {"line": 14, "column": 7}}],
 "messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 7, "column": 4},
   "endPos": null,
   "data": "expected token"},
  {"severity": "error",
   "pos": {"line": 13, "column": 19},
   "endPos": {"line": 13, "column": 20},
   "data":
   "failed to synthesize instance of type class\n  OfNat ℝ 1\nnumerals are polymorphic in Lean, but the numeral `1` cannot be used in a context where the expected type is\n  ℝ\ndue to the absence of the instance above\n\nHint: Type class instance resolution failures can be inspected with the `set_option trace.Meta.synthInstance true` command."},
  {"severity": "error",
   "pos": {"line": 11, "column": 31},
   "endPos": {"line": 11, "column": 39},
   "data":
   "Function expected at\n  Finset\nbut this term has type\n  ?m.1\n\nNote: Expected a fun

---

## 4. Le Core

Le **Core** d'un jeu cooperatif est l'ensemble des allocations stables : aucune coalition ne peut faire mieux en se separant.

In [8]:
-- Le Core d'un jeu cooperatif

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une allocation est un vecteur de paiements
def Allocation (N : Type*) := N → ℝ

-- Le Core : allocations efficaces et stables
def Core (G : TUGame N) : Set (Allocation N) :=
  { x | 
    -- Efficacite : la somme = valeur de N
    (∑ i, x i = G.v Finset.univ) ∧
    -- Rationalite de groupe : aucune coalition ne peut bloquer
    (∀ S : Finset N, ∑ i in S, x i ≥ G.v S)
  }

-- Le Core peut etre vide !
def CoreEmpty (G : TUGame N) : Prop := Core G = ∅

#check @Core
#check @CoreEmpty

-- Le Core d'un jeu cooperatif

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Une allocation est un vecteur de paiements
def Allocation (N : Type*) := N → ℝ
                         ─▶ ❌ unexpected token ')'; expected term

-- Le Core : allocations efficaces et stables
def Core (G : TUGame N) : Set (Allocation N) :=
              ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
                          ──────────────────▶ ❌ Function expected at
  Set
but this term has type
  ?m.2

Note: Expected a function because this term is being applied to the argument
  (Allocation N)

Hint: The identifier `Set` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  { x | 
    -- Efficacite : la somme = valeur de N
    (∑ i, x i = G.v Finset.univ) ∧
─────▶ ❌ expected token
    -- Rationalite de groupe : aucune coalition ne peut bloquer
    (∀ S : Finset N, ∑ i in S, x i ≥ G.v S)
  }

-- Le Core peut etre vide !
def CoreEmpty (G : TUGame N) : Prop := Core G = ∅
                   ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.

#check @Core
──────▶  @Core : {x : Sort u_1} → {TUGame : x} → {x : Sort u_2} → {Set : x} → sorry → sorry
#check @CoreEmpty
──────▶  @CoreEmpty : {x : Sort u_1} → {TUGame : x} → sorry → Prop
--% env 7

Raw input:
{"cmd": "-- Le Core d'un jeu cooperatif\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Une allocation est un vecteur de paiements\ndef Allocation (N : Type*) := N \u2192 \u211d\n\n-- Le Core : allocations efficaces et stables\ndef Core (G : TUGame N) : Set (Allocation N) :=\n  { x | \n    -- Efficacite : la somme = valeur de N\n    (\u2211 i, x i = G.v Finset.univ) \u2227\n    -- Rationalite de groupe : aucune coalition ne peut bloquer\n    (\u2200 S : Finset N, \u2211 i in S, x i \u2265 G.v S)\n  }\n\n-- Le Core peut etre vide !\ndef CoreEmpty (G : TUGame N) : Prop := Core G = \u2205\n\n#check @Core\n#check @CoreEmpty", "env": 6}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 6, "column": 25},
   "endPos": {"line": 6, "column": 26},
   "data": "unexpected token ')'; expected term"},
  {"severity": "error",
   "pos": {"line": 12, "column": 5},
   "endPos": null,
   "data": "expected token"},
  {"severity": "error",
   "pos": {"line": 9, "column": 14},
   "endPos": {"line": 9, "column": 22},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown 

Le Core est défini comme l'ensemble des allocations **efficaces** (somme = v(N)) et **stables** (aucune coalition ne peut bloquer).

Le théorème suivant caractérise quand le Core est non-vide.

In [9]:
-- Theoreme de Bondareva-Shapley
-- Le Core est non-vide ssi le jeu est equilibre

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Collection equilibree (balanced collection)
def Balanced (weights : Finset N → ℝ) : Prop :=
  (∀ S, weights S ≥ 0) ∧
  (∀ i : N, ∑ S in (Finset.univ.filter (fun S => i ∈ S)), weights S = 1)

-- Jeu equilibre
def BalancedGame (G : TUGame N) : Prop :=
  ∀ weights : Finset N → ℝ,
    Balanced weights →
    ∑ S in Finset.univ, weights S * G.v S ≤ G.v Finset.univ

-- Theoreme de Bondareva-Shapley
theorem bondareva_shapley (G : TUGame N) :
    (Core G).Nonempty ↔ BalancedGame G := by
  sorry

#check @bondareva_shapley

-- Theoreme de Bondareva-Shapley
-- Le Core est non-vide ssi le jeu est equilibre

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Collection equilibree (balanced collection)
def Balanced (weights : Finset N → ℝ) : Prop :=
                        ────────▶ ❌ Function expected at
  Finset
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Finset` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  (∀ S, weights S ≥ 0) ∧
  (∀ i : N, ∑ S in (Finset.univ.filter (fun S => i ∈ S)), weights S = 1)
────────────▶ ❌ expected token

-- Jeu equilibre
def BalancedGame (G : TUGame N) : Prop :=
                      ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ weights : Finset N → ℝ,
    Balanced weights →
    ∑ S in Finset.univ, weights S * G.v S ≤ G.v Finset.univ
────▶ ❌ expected token

-- Theoreme de Bondareva-Shapley
theorem bondareva_shapley (G : TUGame N) :
                               ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    (Core G).Nonempty ↔ BalancedGame G := by
  sorry

#check @bondareva_shapley
        ─────────────────▶ ❌ Unknown identifier `bondareva_shapley`
--% env 8
--% prove 2

Raw input:
{"cmd": "-- Theoreme de Bondareva-Shapley\n-- Le Core est non-vide ssi le jeu est equilibre\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Collection equilibree (balanced collection)\ndef Balanced (weights : Finset N \u2192 \u211d) : Prop :=\n  (\u2200 S, weights S \u2265 0) \u2227\n  (\u2200 i : N, \u2211 S in (Finset.univ.filter (fun S => i \u2208 S)), weights S = 1)\n\n-- Jeu equilibre\ndef BalancedGame (G : TUGame N) : Prop :=\n  \u2200 weights : Finset N \u2192 \u211d,\n    Balanced weights \u2192\n    \u2211 S in Finset.univ, weights S * G.v S \u2264 G.v Finset.univ\n\n-- Theoreme de Bondareva-Shapley\ntheorem bondareva_shapley (G : TUGame N) :\n    (Core G).Nonempty \u2194 BalancedGame G := by\n  sorry\n\n#check @bondareva_shapley", "env": 7}
Raw output:
{"sorries":
 [{"proofState": 2,
   "pos": {"line": 20, "column": 2},
   "goal":
   "x✝ : Sort u_1\nTUGame : x✝\nG : sorry\n⊢ sorry ↔ @BalancedGame (?m.12 G) (?m.13 G) G",
   "endPos": {"line": 20, "column": 7}}],
 "messages":
 [{"severity": "error",
   "pos": {"line": 4, "column": 19},
   "endPos": {"line": 4, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 9, "column": 12},
   "endPos": null,
   "data": "expected token"},
  {"severity": "error",
   "pos": {"line": 7, "column": 24},
   "endPos": {"line": 7, "column": 32},
   "data":
   "Function expected at\n  Finset\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The i

Le théorème de **Bondareva-Shapley** utilise la notion de "couverture équilibrée" pour caractériser la non-vacuité du Core.

Pour les jeux **convexes** (où les contributions marginales croissent), on a un résultat plus fort : la valeur de Shapley est toujours dans le Core.

In [10]:
-- Pour les jeux convexes, Shapley est dans le Core

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem convex_shapley_in_core (G : TUGame N) (h : Convex G) :
    (fun i => shapleyValue G i) ∈ Core G := by
  sorry

#check @convex_shapley_in_core

-- Pour les jeux convexes, Shapley est dans le Core

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

theorem convex_shapley_in_core (G : TUGame N) (h : Convex G) :
                                    ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    (fun i => shapleyValue G i) ∈ Core G := by
────────────────────────────────────────▶ ❌ stuck at solving universe constraint
  ?u.5787+1 =?= imax ?u.5830 ?u.5834
while trying to unify
  (i : ?m.14) → ?m.22 i : Sort (imax ?u.5830 ?u.5834)
with
  (i : ?m.14) → ?m.22 i : Sort (imax ?u.5830 ?u.5834)
  sorry

#check @convex_shapley_in_core
        ──────────────────────▶ ❌ Unknown identifier `convex_shapley_in_core`
--% env 9

Raw input:
{"cmd": "-- Pour les jeux convexes, Shapley est dans le Core\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\ntheorem convex_shapley_in_core (G : TUGame N) (h : Convex G) :\n    (fun i => shapleyValue G i) \u2208 Core G := by\n  sorry\n\n#check @convex_shapley_in_core", "env": 8}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 5, "column": 36},
   "endPos": {"line": 5, "column": 44},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement."},
  {"severity": "error",
   "pos": {"line": 5, "column": 0},
   "endPos": {"line": 6, "column": 40},
   "data":
   "stuck at solving universe constraint\n  ?u.5787+1 =?= imax ?u.5830 ?u.5834\nwhile trying to unify\n  (i : ?m.14) → ?m.22 i : Sort (imax ?u.5830 ?u.5834)\nwith\n  (i : ?m.14) → ?m.22 i : Sort (imax ?u.5830 ?u.5834)"},
  {"severity": "error",
   "pos": {"line": 9, "column": 8},
   "endPos": {"line": 9, "column": 30},
   "data": "Unknown identifier `convex_shapley_in_core`"}],
 "env": 9}

---

## 5. Jeux de Vote

Les jeux de vote sont des cas particuliers ou $v(S) \in \{0, 1\}$ (une coalition gagne ou perd).

In [11]:
-- Jeux de vote ponderes

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Jeu de vote pondere [q; w1, w2, ..., wn]
def WeightedVotingGame (weights : N → ℝ) (quota : ℝ) : TUGame N := {
  v := fun S => if ∑ i in S, weights i ≥ quota then 1 else 0
  empty_zero := by simp
}

-- Un joueur est critique dans une coalition si son retrait la fait perdre
def Critical (G : TUGame N) (i : N) (S : Finset N) : Prop :=
  i ∈ S ∧ G.v S = 1 ∧ G.v (S.erase i) = 0

-- Indice de Banzhaf (non normalise)
def BanzhafRaw (G : TUGame N) (i : N) : ℕ :=
  (Finset.univ.filter (fun S => Critical G i S)).card

#check @WeightedVotingGame
#check @Critical
#check @BanzhafRaw

-- Jeux de vote ponderes

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Jeu de vote pondere [q; w1, w2, ..., wn]
def WeightedVotingGame (weights : N → ℝ) (quota : ℝ) : TUGame N := {
                                                       ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.3

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  v := fun S => if ∑ i in S, weights i ≥ quota then 1 else 0
───────────────────▶ ❌ expected token
  empty_zero := by simp
}

-- Un joueur est critique dans une coalition si son retrait la fait perdre
def Critical (G : TUGame N) (i : N) (S : Finset N) : Prop :=
                  ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
                                         ────────▶ ❌ Function expected at
  Finset
but this term has type
  ?m.3

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `Finset` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  i ∈ S ∧ G.v S = 1 ∧ G.v (S.erase i) = 0

-- Indice de Banzhaf (non normalise)
def BanzhafRaw (G : TUGame N) (i : N) : ℕ :=
                    ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  (Finset.univ.filter (fun S => Critical G i S)).card
   ──────────────────▶ ❌ Unknown identifier `Finset.univ.filter`

#check @WeightedVotingGame
──────▶  @WeightedVotingGame : {N : Sort u_1} → {ℝ : Sort u_2} → {x : Sort u_3} → {TUGame : x} → (N → ℝ) → ℝ → sorry
#check @Critical
──────▶  @Critical : {x : Sort u_1} → {TUGame : x} → {N : Type u_2} → {x : Sort u_3} → {Finset : x} → sorry → N → sorry → Prop
#check @BanzhafRaw
──────▶  @BanzhafRaw : {x : Sort u_1} → {TUGame : x} → {N : Sort u_2} → {ℕ : Sort u_3} → sorry → N → ℕ
--% env 10

Raw input:
{"cmd": "-- Jeux de vote ponderes\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Jeu de vote pondere [q; w1, w2, ..., wn]\ndef WeightedVotingGame (weights : N \u2192 \u211d) (quota : \u211d) : TUGame N := {\n  v := fun S => if \u2211 i in S, weights i \u2265 quota then 1 else 0\n  empty_zero := by simp\n}\n\n-- Un joueur est critique dans une coalition si son retrait la fait perdre\ndef Critical (G : TUGame N) (i : N) (S : Finset N) : Prop :=\n  i \u2208 S \u2227 G.v S = 1 \u2227 G.v (S.erase i) = 0\n\n-- Indice de Banzhaf (non normalise)\ndef BanzhafRaw (G : TUGame N) (i : N) : \u2115 :=\n  (Finset.univ.filter (fun S => Critical G i S)).card\n\n#check @WeightedVotingGame\n#che

Les jeux de vote pondérés sont caractérisés par un **quota** et des **poids**. Une coalition gagne si la somme de ses poids atteint le quota.

Le lemme suivant identifie les joueurs "dummy" (qui n'apportent rien) et les joueurs "dictateurs" (qui peuvent gagner seuls).

In [12]:
-- Proprietes des joueurs dans les jeux de vote

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Joueur avec droit de veto
def VetoPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, G.v S = 1 → i ∈ S

-- Dictateur
def Dictator (G : TUGame N) (i : N) : Prop :=
  G.v {i} = 1 ∧ VetoPlayer G i

-- Joueur nul (dummy)
def DummyPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, i ∉ S → G.v (S ∪ {i}) = G.v S

-- Un joueur nul a Shapley = 0
theorem dummy_shapley_zero (G : TUGame N) (i : N) :
    DummyPlayer G i → shapleyValue G i = 0 := by
  sorry

#check @VetoPlayer
#check @Dictator
#check @DummyPlayer
#check @dummy_shapley_zero

-- Proprietes des joueurs dans les jeux de vote

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

-- Joueur avec droit de veto
def VetoPlayer (G : TUGame N) (i : N) : Prop :=
                    ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S : Finset N, G.v S = 1 → i ∈ S
        ──────▶ ❌ Unknown identifier `Finset`

-- Dictateur
def Dictator (G : TUGame N) (i : N) : Prop :=
                  ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  G.v {i} = 1 ∧ VetoPlayer G i

-- Joueur nul (dummy)
def DummyPlayer (G : TUGame N) (i : N) : Prop :=
                     ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  ∀ S : Finset N, i ∉ S → G.v (S ∪ {i}) = G.v S
        ──────▶ ❌ Unknown identifier `Finset`

-- Un joueur nul a Shapley = 0
theorem dummy_shapley_zero (G : TUGame N) (i : N) :
                                ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    DummyPlayer G i → shapleyValue G i = 0 := by
  sorry

#check @VetoPlayer
──────▶  @VetoPlayer : {x : Sort u_1} → {TUGame : x} → {N : Type u_2} → sorry → N → Prop
#check @Dictator
──────▶  @Dictator : {x : Sort u_1} → {TUGame : x} → {N : Type u_2} → sorry → N → Prop
#check @DummyPlayer
──────▶  @DummyPlayer : {x : Sort u_1} → {TUGame : x} → {N : Type u_2} → sorry → N → Prop
#check @dummy_shapley_zero
        ──────────────────▶ ❌ Unknown identifier `dummy_shapley_zero`
--% env 11
--% prove 3

Raw input:
{"cmd": "-- Proprietes des joueurs dans les jeux de vote\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\n-- Joueur avec droit de veto\ndef VetoPlayer (G : TUGame N) (i : N) : Prop :=\n  \u2200 S : Finset N, G.v S = 1 \u2192 i \u2208 S\n\n-- Dictateur\ndef Dictator (G : TUGame N) (i : N) : Prop :=\n  G.v {i} = 1 \u2227 VetoPlayer G i\n\n-- Joueur nul (dummy)\ndef DummyPlayer (G : TUGame N) (i : N) : Prop :=\n  \u2200 S : Finset N, i \u2209 S \u2192 G.v (S \u222a {i}) = G.v S\n\n-- Un joueur nul a Shapley = 0\ntheorem dummy_shapley_zero (G : TUGame N) (i : N) :\n    DummyPlayer G i \u2192 shapleyValue G i = 0 := by\n  sorry\n\n#check @VetoPlayer\n#check @Dictator\n#check @DummyPlayer\n#check @dummy_shapley_zero", "env": 10}
Raw output:
{"sorri

---

## 6. Exercices

### Exercice 1 : Jeu de gants

Trois joueurs : L1, L2 ont chacun un gant gauche, R1 a un gant droit. Une paire de gants vaut 1.

1. Definir la fonction caracteristique
2. Calculer la valeur de Shapley de chaque joueur

### Exercice 2 : Verification d'axiome

Prouver que la valeur de Shapley satisfait l'axiome du joueur nul.

### Exercice 3 : Core vide

Montrer que le jeu de majorite simple a 3 joueurs a un Core vide.

---

## 7. Solutions

### Solution Exercice 1

In [13]:
-- Solution Exercice 1 : Jeu de gants

-- Trois joueurs : L1, L2 (gants gauches), R1 (gant droit)
inductive GlovePlayer where
  | L1 | L2 | R1
  deriving DecidableEq, Fintype

open GlovePlayer

def hasLeftGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => true
  | L2 => true
  | R1 => false

def hasRightGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => false
  | L2 => false
  | R1 => true

-- Fonction caracteristique : nombre de paires completes
def gloveGame : TUGame GlovePlayer := {
  v := fun S => 
    let nLeft := (S.filter (fun p => hasLeftGlove p)).card
    let nRight := (S.filter (fun p => hasRightGlove p)).card
    min nLeft nRight
  empty_zero := by simp
}

-- Valeurs de Shapley (calcul manuel) :
-- R1 a une contribution marginale plus elevee (ressource rare)
-- Shapley(L1) = Shapley(L2) = 1/6
-- Shapley(R1) = 4/6 = 2/3

#check gloveGame

-- Solution Exercice 1 : Jeu de gants

-- Trois joueurs : L1, L2 (gants gauches), R1 (gant droit)
inductive GlovePlayer where
  | L1 | L2 | R1
  deriving DecidableEq, Fintype
                        ───────▶ ❌ Unknown constant `Fintype`

open GlovePlayer

def hasLeftGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => true
  | L2 => true
  | R1 => false

def hasRightGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => false
  | L2 => false
  | R1 => true

-- Fonction caracteristique : nombre de paires completes
def gloveGame : TUGame GlovePlayer := {
                ──────────────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  GlovePlayer

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
  v := fun S => 
    let nLeft := (S.filter (fun p => hasLeftGlove p)).card
    let nRight := (S.filter (fun p => hasRightGlove p)).card
    min nLeft nRight
  empty_zero := by simp
}

-- Valeurs de Shapley (calcul manuel) :
-- R1 a une contribution marginale plus elevee (ressource rare)
-- Shapley(L1) = Shapley(L2) = 1/6
-- Shapley(R1) = 4/6 = 2/3

#check gloveGame
──────▶  gloveGame.{u_1, u_2} {x✝ : Sort u_1} {TUGame : x✝} : sorry
--% env 12

Raw input:
{"cmd": "-- Solution Exercice 1 : Jeu de gants\n\n-- Trois joueurs : L1, L2 (gants gauches), R1 (gant droit)\ninductive GlovePlayer where\n  | L1 | L2 | R1\n  deriving DecidableEq, Fintype\n\nopen GlovePlayer\n\ndef hasLeftGlove (p : GlovePlayer) : Bool :=\n  match p with\n  | L1 => true\n  | L2 => true\n  | R1 => false\n\ndef hasRightGlove (p : GlovePlayer) : Bool :=\n  match p with\n  | L1 => false\n  | L2 => false\n  | R1 => true\n\n-- Fonction caracteristique : nombre de paires completes\ndef gloveGame : TUGame GlovePlayer := {\n  v := fun S => \n    let nLeft := (S.filter (fun p => hasLeftGlove p)).card\n    let nRight := (S.filter (fun p => hasRightGlove p)).card\n    min nLeft nRight\n  empty_zero := by simp\n}\n\n-- Valeurs de Shapley (calcul manuel) :\n-- R1 a une contribution marginale plus elevee (ressource rare)\n-- Shapley(L1) = Shapley(L2) = 1/6\n-- Shapley(R1) = 4/6 = 2/3\n\n#check gloveGame", "env": 11}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 6, "column": 24},
   "endPos": {"line": 6, "column": 31},
   "data": "Unknown constant `Fintype`"},
  {"severity": "error",
   "pos": {"line": 23, "column": 16},
   "endPos": {"line": 23, "column": 34},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  GlovePlayer\n\nHint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement."},
  {"severity": "info",
   "pos": {"line": 36, "column": 0},
   "endPos": {"line": 36, "column": 6},
   "data": "gloveGame.{u_1, u_2} {x✝ : Sort u_1} {TUGame : x✝} : sorry"}],
 "env": 12}

### Solution Exercice 2

In [14]:
-- Solution Exercice 2 : Shapley satisfait l'axiome du joueur nul

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem shapley_null_player (G : TUGame N) (i : N) :
    NullPlayer G i → shapleyValue G i = 0 := by
  intro h_null
  -- Si i est un joueur nul, sa contribution marginale est toujours 0
  unfold shapleyValue
  apply Finset.sum_eq_zero
  intro S hS
  simp at hS
  have h : marginalContribution G i S = 0 := by
    unfold marginalContribution
    rw [h_null S hS]
    ring
  simp [h]

#check @shapley_null_player

-- Solution Exercice 2 : Shapley satisfait l'axiome du joueur nul

variable {N : Type*} [Fintype N] [DecidableEq N]
                   ─▶ ❌ unexpected token '}'; expected term

theorem shapley_null_player (G : TUGame N) (i : N) :
                                 ────────▶ ❌ Function expected at
  TUGame
but this term has type
  ?m.1

Note: Expected a function because this term is being applied to the argument
  N

Hint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement.
    NullPlayer G i → shapleyValue G i = 0 := by
  intro h_null
  -- Si i est un joueur nul, sa contribution marginale est toujours 0
  unfold shapleyValue
  apply Finset.sum_eq_zero
        ──────────────────▶ ❌ Unknown identifier `Finset.sum_eq_zero`
  intro S hS
  simp at hS
  have h : marginalContribution G i S = 0 := by
    unfold marginalContribution
    rw [h_null S hS]
    ring
─────▶ ❌ unknown tactic
  simp [h]

#check @shapley_null_player
        ───────────────────▶ ❌ Unknown identifier `shapley_null_player`
--% env 13

Raw input:
{"cmd": "-- Solution Exercice 2 : Shapley satisfait l'axiome du joueur nul\n\nvariable {N : Type*} [Fintype N] [DecidableEq N]\n\ntheorem shapley_null_player (G : TUGame N) (i : N) :\n    NullPlayer G i \u2192 shapleyValue G i = 0 := by\n  intro h_null\n  -- Si i est un joueur nul, sa contribution marginale est toujours 0\n  unfold shapleyValue\n  apply Finset.sum_eq_zero\n  intro S hS\n  simp at hS\n  have h : marginalContribution G i S = 0 := by\n    unfold marginalContribution\n    rw [h_null S hS]\n    ring\n  simp [h]\n\n#check @shapley_null_player", "env": 12}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 3, "column": 19},
   "endPos": {"line": 3, "column": 20},
   "data": "unexpected token '}'; expected term"},
  {"severity": "error",
   "pos": {"line": 16, "column": 5},
   "endPos": null,
   "data": "unknown tactic"},
  {"severity": "error",
   "pos": {"line": 5, "column": 33},
   "endPos": {"line": 5, "column": 41},
   "data":
   "Function expected at\n  TUGame\nbut this term has type\n  ?m.1\n\nNote: Expected a function because this term is being applied to the argument\n  N\n\nHint: The identifier `TUGame` is unknown, and Lean's `autoImplicit` option causes an unknown identifier to be treated as an implicitly bound variable with an unknown type. However, the unknown type cannot be a function, and a function is what Lean expects here. This is often the result of a typo or a missing `import` or `open` statement."},
  {"severity": "error",
   "pos": {"line": 10, "column": 8},
   "endPos": {"line": 10, "column": 26},
   "data": "Unknown identifier `Finset.sum_eq_zero`"},
  {"severity": "error",
   "pos": {"line": 19, "column": 8},
   "endPos": {"line": 19, "column": 27},
   "data": "Unknown identifier `shapley_null_player`"}],
 "env": 13}

### Solution Exercice 3

Pour la preuve que le Core du jeu de majorite a 3 joueurs est vide, voir le notebook Python compagnon (21b) pour l'illustration numerique.

---

## Resume

### Concepts formalises

| Concept | Definition Lean |
|---------|----------------|
| TU Game | `TUGame N` avec `v : Finset N → ℝ`, `v(∅) = 0` |
| Superadditivite | `v(S ∪ T) ≥ v(S) + v(T)` pour S, T disjoints |
| Convexite | Contributions marginales croissantes |
| Shapley | Contribution marginale moyenne sur permutations |
| Core | Allocations efficaces et stables |
| Banzhaf | Nombre de coalitions critiques |

### Axiomes de Shapley

| Axiome | Signification |
|--------|---------------|
| Efficacite | `∑ φ = v(N)` |
| Symetrie | Joueurs interchangeables = meme paiement |
| Joueur nul | Contribution nulle = paiement nul |
| Additivite | `φ(G+H) = φ(G) + φ(H)` |

### Theoremes cles

1. **Unicite de Shapley** : Les 4 axiomes caracterisent Shapley uniquement
2. **Bondareva-Shapley** : Core non-vide ⇔ jeu equilibre
3. **Jeux convexes** : Shapley dans le Core

---

## Conclusion de la Serie GameTheory

Cette serie de **21 notebooks** (+3 compagnons Python) a couvert la theorie des jeux de maniere complete :

| Partie | Focus |
|--------|-------|
| 1-6 | Fondations : Nash, minimax, evolution |
| 7-11 | Dynamique : forme extensive, induction, bayesien |
| 12-16 | Algorithmes : CFR, cooperatif, mecanismes, MARL |
| 17-21 | Formalisation : Lean 4, preuves, Shapley |

---

**Navigation** : [← GameTheory-16b-Lean-SocialChoice](GameTheory-16b-Lean-SocialChoice.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]